In [10]:
from significance_analysis import *
from pymer4.models import Lmer
import bambi
import arviz as az
import pandas as pd

In [11]:
data = pd.read_parquet("significance_analysis_example/datasets/priorband_data.parquet")
data=data.loc[(data["seed"]==1) & (data["used_fidelity"]==10)]

In [12]:
lmem=Lmer("value ~ algorithm + (1|benchmark)",data)
lmem.fit(factors={"algorithm":list(data["algorithm"].unique())})
print(lmem.summary())
print(lmem.post_hoc(marginal_vars=["algorithm"]))#,grouping_vars=["benchmark"])[0])

c:\Users\lenak\OFFLINE_FILES\Anton\Git\lmem-significance\.venv_3_10_0\lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: value~algorithm+(1|benchmark)

Family: gaussian	 Inference: parametric

Number of observations: 72	 Groups: {'benchmark': 12.0}

Log-likelihood: -39.873 	 AIC: 89.746

Random effects:

                  Name    Var    Std
benchmark  (Intercept)  9.887  3.144
Residual                0.053  0.230

No random effect correlations specified

Fixed effects:

Linear mixed model fit by REML [’lmerMod’]
Formula: value~algorithm+(1|benchmark)

Family: gaussian	 Inference: parametric

Number of observations: 72	 Groups: {'benchmark': 12.0}

Log-likelihood: -39.873 	 AIC: 89.746

Random effects:

                  Name    Var    Std
benchmark  (Intercept)  9.887  3.144
Residual                0.053  0.230

No random effect correlations specified

Fixed effects:

             Estimate  2.5_ci  97.5_ci     SE      DF  T-stat  P-val Sig
(Intercept)     1.891   0.110    3.673  0.909  11.039   2.081  0.062   .
algorithm1      0.082  -0.048    0.212  0.

In [13]:
bamb=bambi.Model("value ~ algorithm + (1|benchmark)",data)
print(bamb)
results=bamb.fit()
print(results)


       Formula: value ~ algorithm + (1|benchmark)
        Family: gaussian
          Link: mu = identity
  Observations: 72
        Priors: 
    target = mu
        Common-level effects
            Intercept ~ Normal(mu: 1.9113, sigma: 10.6744)
            algorithm ~ Normal(mu: [0. 0.], sigma: [16.0117 16.0117])
        
        Group-level effects
            1|benchmark ~ Normal(mu: 0.0, sigma: HalfNormal(sigma: 10.6744))
        
        Auxiliary parameters
            sigma ~ HalfStudentT(nu: 4.0, sigma: 3.0192)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, algorithm, 1|benchmark_sigma, 1|benchmark_offset]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 3444 seconds.
There were 14 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data


In [14]:
az.summary(results)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
1|benchmark[JAHS-C10],7.944,1.075,5.863,9.967,0.046,0.033,550.0,607.0,1.01
1|benchmark[JAHS-CH],3.082,1.076,1.065,5.216,0.046,0.034,556.0,584.0,1.01
1|benchmark[JAHS-FM],3.347,1.075,1.281,5.399,0.046,0.034,552.0,624.0,1.01
1|benchmark[LC-126026],-1.903,1.076,-3.950,0.179,0.046,0.033,552.0,579.0,1.01
1|benchmark[LC-167190],-1.757,1.074,-3.860,0.280,0.046,0.033,556.0,595.0,1.01
1|benchmark[LC-168330],-1.538,1.077,-3.585,0.574,0.046,0.033,558.0,579.0,1.01
1|benchmark[LC-168910],-1.643,1.076,-3.726,0.428,0.046,0.033,555.0,600.0,1.01
1|benchmark[LC-189906],-1.816,1.077,-3.920,0.220,0.046,0.033,558.0,579.0,1.01
1|benchmark[PD1-Cifar100],-1.676,1.077,-3.674,0.465,0.046,0.033,559.0,590.0,1.01
1|benchmark[PD1-ImageNet],-1.681,1.075,-3.688,0.432,0.046,0.033,555.0,587.0,1.01
